# Analyze single sample

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re 
import os
import sys
import seaborn as sns
sns.set_theme(style="whitegrid")

In [ ]:
data_folder = "../dataset"
test_id = 'ismb2025'

In [4]:
def read_score_to_tuple(sample, score, input_type, mix):
    cols = ["sample", "species", "model", "bintype", "tool", "prec", "rec", "f1"]
    out_tuple = pd.DataFrame(columns=cols)

    # out_tuple = pd.DataFrame(columns=cols)
    # voglio leggere i file .txt del typo "score" (lab o bin) e del tipo "input_type" (ske, uni, pan)
    # e metterli in un dataframe
    # se input type == pan allora devo fare la "media" dei valori di ske e uni
    # riguardo al tipo di bin: uni e ske hanno solo pred, mentre pan ha anche naive e graph-overlap
    for filename in os.listdir(sample):
        fields = filename.split(".")
        if (not filename.endswith(".txt")): # salto i file non txt, non sono file di score
            continue
        _species = "null"
        _sample = fields[0] #nome del sample
        _type = fields[1] # tipo in input {ske, uni, pan}
        _thr = fields[2] # threshold (default 1)
        _model = fields[3] # modello {pbf, ml}, default pbf
        assert(_model == "pbf")
        _score = fields[-2] # score {lab, bin} labeling o binning
        _ref = fields[-3] # reference {uni, ske}
        _bins = fields[-4] # tipo di binning {pred, nve, ovl} pred= predizione del modello, nve= naive, ovl= graph-overlap
        
        if _type == input_type:
            if _score == "lab" and score == "lab": # se il tipo di score del file è del tipo richiesto
                assert(filename.endswith(".txt"))
                with open(sample+ "/" + filename) as file:
                    lines = file.readlines()
                    prec, rec, f1 = float(lines[-4].split("\t")[-1].strip()), float(lines[-3].split("\t")[-1].strip()), float(lines[-2].split("\t")[-1].strip())

            elif _score == "bin" and score == "bin":
                assert(filename.endswith(".txt"))
                with open(sample+ "/" + filename) as file:
                    lines = file.readlines()
                    prec, rec, f1 = float(lines[-3].split("\t")[-1].strip()), float(lines[-2].split("\t")[-1].strip()), float(lines[-1].split("\t")[-1].strip())
            else:
                continue
            out_tuple = pd.concat([pd.DataFrame([[_sample, _species, _model, _bins, f"{_thr}.{_type}", prec, rec, f1]], columns=cols), out_tuple if not out_tuple.empty else None], ignore_index=True)

        else:
            continue
        # print(out_tuple)
        if len(out_tuple) > 1:
            out_tuple = out_tuple.groupby(["sample", "model", "species", "bintype", "tool"]).mean(numeric_only=True).reset_index()
            # out_tuple.insert(loc=3, column="tool", value = f"{_thr}.{_type}")
            if (mix):
                out_tuple = out_tuple.groupby(["sample", "model", "species", "tool"]).max(numeric_only=True).reset_index()
                out_tuple.insert(loc=2, column="bintype", value = f"pred")
                # out_tuple.insert(loc=3, column="tool", value = f"{_thr}.{_type}")
            

    return out_tuple

In [5]:
def read_gplas_to_tuple(filename, sample, species, type, score, zero):
    _model = "gplas"
    _species = species
    _sample = sample
    _type = type # uni, ske
    _thr = 1
    _score = score # lab, bin
    _ref = type
    _bins = "pred"
    filename = filename # SAMNXXXXX-u.gplas.bin.txt
    cols = ["sample", "species", "model", "bintype", "tool", "prec", "rec", "f1"]
    out_tuple = pd.DataFrame(columns=cols)
    fields = filename.split(".")
    if (not filename.endswith(".txt")): # salto i file non txt, non sono file di score
        return None
    if _score == "lab":
        if zero:
            prec, rec, f1 = 0, 0, 0
        else:   
            with open(filename) as file:
                    lines = file.readlines()
                    prec, rec, f1 = float(lines[-4].split("\t")[-1].strip()), float(lines[-3].split("\t")[-1].strip()), float(lines[-2].split("\t")[-1].strip())
    elif _score == "bin":
        if zero:
            prec, rec, f1 = 0, 0, 0
        else:
            with open(filename) as file:
                    lines = file.readlines()
                    prec, rec, f1 = float(lines[-3].split("\t")[-1].strip()), float(lines[-2].split("\t")[-1].strip()), float(lines[-1].split("\t")[-1].strip())
                    
    else:
        return None
    out_tuple = pd.concat([pd.DataFrame([[_sample, _species, _model, _bins, f"{_thr}.{_type}", prec, rec, f1]], columns=cols), out_tuple if not out_tuple.empty else None], ignore_index=True)


    assert len(out_tuple) == 1
    return out_tuple

In [ ]:
#iterate over all samples
df_labeling = pd.DataFrame(columns=["sample", "model", "bintype", "tool", "prec", "rec", "f1"])
df_labeling_mix = pd.DataFrame(columns=["sample", "model", "bintype", "tool", "prec", "rec", "f1"])
df_binning = pd.DataFrame(columns=["sample", "model", "bintype", "tool", "prec", "rec", "f1"])
df_binning_mix = pd.DataFrame(columns=["sample", "model", "bintype", "tool", "prec", "rec", "f1"])
for file in os.listdir(data_folder):
    sample = data_folder + file
    for input_type in ["ske", "uni", "pan"]:
        df_labeling = pd.concat([read_score_to_tuple(sample, "lab", input_type, mix=False), df_labeling if not df_labeling.empty else None], ignore_index=True)
        df_labeling_mix = pd.concat([read_score_to_tuple(sample, "lab", input_type, mix=True), df_labeling_mix if not df_labeling_mix.empty else None], ignore_index=True)

        df_binning = pd.concat([read_score_to_tuple(sample, "bin", input_type, mix=False), df_binning if not df_binning.empty else None], ignore_index=True)
        df_binning_mix = pd.concat([read_score_to_tuple(sample, "bin", input_type, mix=True), df_binning_mix if not df_binning_mix.empty else None], ignore_index=True)

gplas_res = "/home/sgro/gplas/results"
gplas_data = "/data/proj/pangebin/data-gplas/"
# def read_gplas_to_tuple(filename, sample, type, score):

for species in os.listdir(gplas_data):
    speciesStr = species.casefold().split(" ")[0]
    for id in os.listdir(f"{gplas_data}/{species}"):
        zero_lab_s = False
        zero_lab_u = False
        zero_bin_s = False
        zero_bin_u = False

        labeling_res_u = f"{gplas_res}/{id}-u.gplas.lab.txt"
        labeling_res_s = f"{gplas_res}/{id}-s.gplas.lab.txt"

        binning_res_u = f"{gplas_res}/{id}-u.gplas.bin.txt"
        binning_res_s = f"{gplas_res}/{id}-s.gplas.bin.txt"
        try:
            assert os.path.exists(labeling_res_u)
        except AssertionError:
            zero_lab_u = True
        try:
            assert os.path.exists(labeling_res_s)
        except AssertionError:
            zero_lab_s = True
        try:
            assert os.path.exists(binning_res_u)
        except AssertionError:
            zero_bin_u = True
        try:
            assert os.path.exists(binning_res_s)
        except AssertionError:
            zero_bin_s = True

        df_labeling = pd.concat([read_gplas_to_tuple(labeling_res_u, f"{id}", speciesStr, "uni", "lab", zero_lab_u), df_labeling if not df_labeling.empty else None], ignore_index=True)
        df_labeling = pd.concat([read_gplas_to_tuple(labeling_res_s, f"{id}", speciesStr, "ske", "lab", zero_lab_s), df_labeling if not df_labeling.empty else None], ignore_index=True)

        df_binning = pd.concat([read_gplas_to_tuple(binning_res_u, f"{id}", speciesStr,  "uni", "bin", zero_bin_u), df_binning if not df_binning.empty else None], ignore_index=True)
        df_binning = pd.concat([read_gplas_to_tuple(binning_res_s, f"{id}", speciesStr, "ske", "bin", zero_bin_s), df_binning if not df_binning.empty else None], ignore_index=True)

        

df_labeling = df_labeling[df_labeling.bintype != "nve"]
df_labeling = df_labeling[df_labeling.bintype != "ovl"]
# taken a sample, replace species with null value with the species from the same sample id
for index, row in df_labeling.iterrows():
    if row["tool"] == "1.pan":
        row["model"] == "pan"
    if row["species"] == "null":
        sample = row["sample"]
        row["species"] = df_labeling[df_labeling["sample"] == sample].species.unique()[0]
    df_labeling.loc[index] = row

df_labeling = df_labeling.sort_values(by=["species", "model", "bintype", "tool"])



df_binning = df_binning[df_binning.bintype != "nve"]
df_binning = df_binning[df_binning.bintype != "ovl"]
for index, row in df_binning.iterrows():
    if row["tool"] == "1.pan":
        row["model"] == "pan"
    if row["species"] == "null":
        sample = row["sample"]
        row["species"] = df_binning[df_binning["sample"] == sample].species.unique()[0]
    df_binning.loc[index] = row

df_binning = df_binning.sort_values(by=["species", "model", "bintype", "tool"])



df_labeling.to_csv(f"results/{test_id}.labeling.csv", sep='\t', index=False)
df_binning.to_csv(f"results/{test_id}.binning.csv", sep='\t', index=False)

df_labeling_mix.to_csv(f"results/{test_id}.labeling.mix.csv", sep='\t', index=False)
df_binning_mix.to_csv(f"results/{test_id}.binning.mix.csv", sep='\t', index=False)



In [ ]:
# Removing samples, acinetobacter and enterococcus because not in the plasmid database

df_lab = df_labeling[df_labeling["species"] != "acinetobacter"]
df_lab = df_lab[df_lab["species"] != "enterococcus"]

df_bin = df_binning[df_binning["species"] != "acinetobacter"]
df_bin = df_bin[df_bin["species"] != "enterococcus"]

# extract ids that have precision, recall and f1 equal to 0 for 1.uni and 1.ske, i.e. solution was empty meaning that the plasmid database matching step provided non plasmids for sample.
ids_u = df_lab.loc[(df_lab["prec"] == 0) & (df_lab["rec"] == 0) & (df_lab["model"]=='pbf') & (df_lab["tool"]=='1.uni'), "sample"].unique()
ids_s = df_lab.loc[(df_lab["prec"] == 0) & (df_lab["rec"] == 0) & (df_lab["model"]=='pbf') & (df_lab["tool"]=='1.ske'), "sample"].unique()

print(len(ids_u))
print(len(ids_s))

remove_set = set(ids_u).intersection(set(ids_s))
print(remove_set)
print(len(remove_set))
# remove_set = remove_set - {'SAMN16357502'}

df_lab_cut = df_lab[~df_lab["sample"].isin(remove_set)]
df_bin_cut = df_bin[~df_bin["sample"].isin(remove_set)]

df_lab_cut.to_csv(f"results/{test_id}.labeling.cut.csv", sep='\t', index=False)
df_bin_cut.to_csv(f"results/{test_id}.binning.cut.csv", sep='\t', index=False)

df_lab_rem = df_lab[df_lab["sample"].isin(remove_set)]
df_bin_rem = df_bin[df_bin["sample"].isin(remove_set)]

df_lab_rem.to_csv(f"results/{test_id}.labeling.rem.csv", sep='\t', index=False)
df_bin_rem.to_csv(f"results/{test_id}.binning.rem.csv", sep='\t', index=False)





7
7
{'SAMN16357205', 'SAMN16357459', 'SAMN16357501', 'SAMN16357460', 'SAMN16357462', 'SAMN01057614', 'SAMN16357502'}
7


In [ ]:
#PLOT LABELING
df_labeling = pd.read_csv(f"results/{test_id}.labeling.cut.csv", sep='\t')
df_binning = pd.read_csv(f"results/{test_id}.binning.cut.csv", sep='\t')
# PLOT GPLAS unicycler vs PBF unicyler
len(df_labeling["sample"].unique())

48

In [9]:
df_labeling[["species"]].value_counts()/5


species    
klebsiella     40.0
escherichia     8.0
Name: count, dtype: float64

In [10]:
import seaborn as sns
from matplotlib.legend_handler import HandlerTuple


plt.

In [11]:
def load_scores(type_):
    assert type_ in ["labeling", "binning"]
    global df_scores, df_pbf_unicycler, df_gplas_unicycler, df_pangebin, df_pbf_skesa, df_gplas_skesa, df_pbf_mean, df_gplas_mean
    df_scores = pd.read_csv(f"{test_id}.{type_}.cut.csv", sep='\t')
    df_pbf_unicycler = df_scores[(df_scores["tool"] == "1.uni") & (df_scores["model"] == "pbf")]
    df_pbf_unicycler = df_pbf_unicycler.groupby(["sample"]).mean(numeric_only=True).reset_index()

    df_gplas_unicycler = df_scores[(df_scores["tool"] == "1.uni") & (df_scores["model"] == "gplas")]
    df_gplas_unicycler = df_gplas_unicycler.groupby(["sample"]).mean(numeric_only=True).reset_index()

    df_pangebin = df_scores[(df_scores["tool"]=="1.pan") & (df_scores["model"] == "pbf")]
    df_pangebin = df_pangebin.groupby(["sample"]).mean(numeric_only=True).reset_index()

    df_pbf_skesa= df_scores[(df_scores["tool"] == "1.ske") & (df_scores["model"] == "pbf")]
    df_pbf_skesa = df_pbf_skesa.groupby(["sample"]).mean(numeric_only=True).reset_index()

    df_gplas_skesa = df_scores[(df_scores["tool"] == "1.ske") & (df_scores["model"] == "gplas")]
    df_gplas_skesa = df_gplas_skesa.groupby(["sample"]).mean(numeric_only=True).reset_index()

    df_pbf_mean = df_scores[(df_scores["tool"] != "1.pan") & (df_scores["model"] == "pbf")]
    df_pbf_mean = df_pbf_mean.groupby(["sample"]).mean(numeric_only=True).reset_index()

    df_gplas_mean = df_scores[(df_scores["tool"] != "1.pan") & (df_scores["model"] == "gplas")]
    df_gplas_mean = df_gplas_mean.groupby(["sample"]).mean(numeric_only=True).reset_index()

In [12]:

def boxplot(df_scores, scoretype):

    df_pbf_unicycler = df_scores[(df_scores["tool"] == "1.uni") & (df_scores["model"] == "pbf")]
    df_pbf_unicycler = df_pbf_unicycler.groupby(["sample"]).mean(numeric_only=True).reset_index()

    df_gplas_unicycler = df_scores[(df_scores["tool"] == "1.uni") & (df_scores["model"] == "gplas")]
    df_gplas_unicycler = df_gplas_unicycler.groupby(["sample"]).mean(numeric_only=True).reset_index()

    df_pangebin = df_scores[(df_scores["tool"]=="1.pan") & (df_scores["model"] == "pbf")]
    df_pangebin = df_pangebin.groupby(["sample"]).mean(numeric_only=True).reset_index()

    df_pbf_skesa= df_scores[(df_scores["tool"] == "1.ske") & (df_scores["model"] == "pbf")]
    df_pbf_skesa = df_pbf_skesa.groupby(["sample"]).mean(numeric_only=True).reset_index()

    df_gplas_skesa = df_scores[(df_scores["tool"] == "1.ske") & (df_scores["model"] == "gplas")]
    df_gplas_skesa = df_gplas_skesa.groupby(["sample"]).mean(numeric_only=True).reset_index()


    df_pbf_mean = df_scores[(df_scores["tool"] != "1.pan") & (df_scores["model"] == "pbf")]
    df_pbf_mean = df_pbf_mean.groupby(["sample"]).mean(numeric_only=True).reset_index()


    header = ["tool", "score", "score_value"]
    df_plot = pd.DataFrame(columns=header)

    df_plot = pd.concat([df_pbf_unicycler[["prec"]].rename(columns={"prec": "score_value"}).assign(tool="pbf_uni").assign(score="prec"), df_plot], ignore_index=True)
    df_plot = pd.concat([
        df_pbf_unicycler[["rec"]].rename(columns={"rec": "score_value"}).assign(tool="pbf_uni").assign(score="rec"), 
        df_plot], ignore_index=True)
    df_plot = pd.concat([
        df_pbf_unicycler[["sample", "f1"]].rename(columns={"f1": "score_value"}).assign(tool="pbf_uni").assign(score="f1"), 
        df_plot], ignore_index=True)

    df_plot = pd.concat([df_pbf_skesa[["sample", "prec"]].rename(columns={"prec": "score_value"}).assign(tool="pbf_ske").assign(score="prec"), df_plot], ignore_index=True)
    df_plot = pd.concat([
        df_pbf_skesa[["sample", "rec"]].rename(columns={"rec": "score_value"}).assign(tool="pbf_ske").assign(score="rec"), 
        df_plot], ignore_index=True)
    df_plot = pd.concat([
        df_pbf_skesa[["sample", "f1"]].rename(columns={"f1": "score_value"}).assign(tool="pbf_ske").assign(score="f1"), 
        df_plot], ignore_index=True)

    df_plot = pd.concat([df_pbf_mean[["sample", "prec"]].rename(columns={"prec": "score_value"}).assign(tool="pbf-mean").assign(score="prec"), df_plot], ignore_index=True)
    df_plot = pd.concat([df_pbf_mean[["sample", "rec"]].rename(columns={"rec": "score_value"}).assign(tool="pbf-mean").assign(score="rec"), df_plot], ignore_index=True)
    df_plot = pd.concat([df_pbf_mean[["sample", "f1"]].rename(columns={"f1": "score_value"}).assign(tool="pbf-mean").assign(score="f1"), df_plot], ignore_index=True)


    df_plot = pd.concat([
        df_gplas_unicycler[["sample", "prec"]].rename(columns={"prec": "score_value"}).assign(tool="gplas_uni").assign(score="prec"), 
        df_plot], ignore_index=True)

    df_plot = pd.concat([
        df_gplas_unicycler[["sample", "rec"]].rename(columns={"rec": "score_value"}).assign(tool="gplas_uni").assign(score="rec"), 
        df_plot], ignore_index=True)
    df_plot = pd.concat([
        df_gplas_unicycler[["sample", "f1"]].rename(columns={"f1": "score_value"}).assign(tool="gplas_uni").assign(score="f1"), 
        df_plot], ignore_index=True) 


    df_plot = pd.concat([
        df_gplas_skesa[["sample", "prec"]].rename(columns={"prec": "score_value"}).assign(tool="gplas_ske").assign(score="prec"), 
        df_plot], ignore_index=True)

    df_plot = pd.concat([
        df_gplas_skesa[["sample", "rec"]].rename(columns={"rec": "score_value"}).assign(tool="gplas_ske").assign(score="rec"), 
        df_plot], ignore_index=True)
    df_plot = pd.concat([
        df_gplas_skesa[["sample", "f1"]].rename(columns={"f1": "score_value"}).assign(tool="gplas_ske").assign(score="f1"), 
        df_plot], ignore_index=True) 

    df_plot = pd.concat([
        df_pangebin[["sample", "prec"]].rename(columns={"prec": "score_value"}).assign(tool="pan").assign(score="prec"), 
        df_plot], ignore_index=True)
    df_plot = pd.concat([
        df_pangebin[["sample", "rec"]].rename(columns={"rec": "score_value"}).assign(tool="pan").assign(score="rec"), 
        df_plot], ignore_index=True)
    df_plot = pd.concat([
        df_pangebin[["sample", "f1"]].rename(columns={"f1": "score_value"}).assign(tool="pan").assign(score="f1"),
        df_plot], ignore_index=True)

    # change header df plot
    df_plot = df_plot.rename(columns={"score_value": "value"})
    # change prec into Precision
    df_plot["score"] = df_plot["score"].replace({"prec": "Precision", "rec": "Recall", "f1": "F1-Score"})

    order = ["gplas_ske", "gplas_uni", "pbf_ske", "pbf_uni", "pbf-mean", "pan"]
    palette = sns.color_palette("tab10", n_colors=6)
    # palette = ["blue", "green", "red", "yellow", "orange", "violet"]
    palette = [palette[0], palette[2], palette[1], palette[3], palette[5], palette[4]]

    plt.figure(figsize=(12, 6))
    p = sns.boxplot(
        data=df_plot,
        x="score",
        order=["Precision", "Recall", "F1-Score"],
        y="value",
        hue="tool",
        hue_order=order,
        boxprops=dict(alpha=.5),
        palette=palette,
    )
    # hide x axis label

    sns.stripplot(data=df_plot, x="score", y="value", hue="tool", hue_order=order,
        dodge=True, ax=p, palette=palette, jitter=0.3).set(xlabel=None, ylabel=None)

    handles, labels = p.get_legend_handles_labels()
    p.legend(handles=[  (handles[0], handles[6]),
                        (handles[1], handles[7]),
                        (handles[2], handles[8]),
                        (handles[3], handles[9]),
                        (handles[4], handles[10]),
                        (handles[5], handles[11])],
            labels=['Gplas (S)', 'Gplas (U)', 'Plasbin-Flow (S)', 'Plasbin-Flow (U)', 'PlasBin-flow (mean)', 'Pangebin'],
            loc='lower left', handlelength=6,
            # handler_map={tuple: HandlerTuple(ndivide=None)})
    )
    sns.move_legend(p, "lower center", bbox_to_anchor=(0.5, -0.15), ncol=6, title=None, frameon=False)


    # p.fig.subplots_adjust(top=.95)
    plt.title(f"{scoretype.upper()} for Gplas, Plasbin-Flow and Pangebin")
    # plt.xlabel("Plasbin-Flow")
    # plt.ylabel("Gplas")
    plt.savefig(f"misc/plot/{scoretype}_whole_2.pdf")
    plt.close()



In [13]:

boxplot(df_binning, "binning")
boxplot(df_labeling, "labeling")



/tmp/ipykernel_970399/3589639385.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_plot = pd.concat([df_pbf_unicycler[["sample", "prec"]].rename(columns={"prec": "score_value"}).assign(tool="pbf_uni").assign(score="prec"), df_plot], ignore_index=True)
/tmp/ipykernel_970399/3589639385.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_plot = pd.concat([df_pbf_unicycler[["sample", "prec"]].rename(columns={"prec": "score_value"}).assign(tool="pbf_uni").assign(score="prec"), df_plot], ign

In [14]:
# compute prec, rec, f1 mean for df_pangebin
def print_scores(df):
    prec = np.round(df["prec"].mean(), 4)
    rec = np.round(df["rec"].mean(), 4)
    f1 = np.round(df["f1"].mean(), 4)
    return float(prec), float(rec), float(f1)


In [15]:
load_scores("labeling")
print(print_scores(df_pangebin))
print(print_scores(df_pbf_mean))
print(print_scores(df_pbf_unicycler))
print(print_scores(df_pbf_skesa))
print(print_scores(df_gplas_unicycler))
print(print_scores(df_gplas_skesa))

(0.7185, 0.917, 0.7643)
(0.6903, 0.9056, 0.7125)
(0.5576, 0.9272, 0.6276)
(0.8229, 0.884, 0.7973)
(0.963, 0.8427, 0.8822)
(0.9059, 0.6399, 0.7214)


In [16]:
load_scores("binning")
print(print_scores(df_pangebin))
print(print_scores(df_pbf_mean))
print(print_scores(df_pbf_unicycler))
print(print_scores(df_pbf_skesa))
print(print_scores(df_gplas_unicycler))
print(print_scores(df_gplas_skesa))

(0.6427, 0.8005, 0.6662)
(0.6721, 0.7952, 0.6581)
(0.5624, 0.8554, 0.6199)
(0.7818, 0.7351, 0.6963)
(0.8427, 0.7764, 0.7785)
(0.8254, 0.5743, 0.6414)


In [17]:
def print_metrics(df):
    prec_m  = np.round(df["prec"].mean(), 3)
    rec_m = np.round(df["rec"].mean(), 3)
    f1_m = np.round(df["f1"].mean(), 3)
    prec_std = np.round(df["prec"].std(), 3)
    rec_std = np.round(df["rec"].std(), 3)
    f1_std = np.round(df["f1"].std(), 3)
    print(f"${prec_m} \\pm {prec_std}$ \t & ${rec_m} \\pm {rec_std}$ \t & ${f1_m} \\pm {f1_std}$ \\\\")


In [18]:
print_metrics(df_pangebin)
print_metrics(df_pbf_mean)
print_metrics(df_pbf_unicycler)
print_metrics(df_pbf_skesa)
print_metrics(df_gplas_unicycler)
print_metrics(df_gplas_skesa)


$0.643 \pm 0.289$ 	 & $0.801 \pm 0.194$ 	 & $0.666 \pm 0.235$ \\
$0.672 \pm 0.247$ 	 & $0.795 \pm 0.199$ 	 & $0.658 \pm 0.212$ \\
$0.562 \pm 0.302$ 	 & $0.855 \pm 0.184$ 	 & $0.62 \pm 0.245$ \\
$0.782 \pm 0.256$ 	 & $0.735 \pm 0.256$ 	 & $0.696 \pm 0.249$ \\
$0.843 \pm 0.209$ 	 & $0.776 \pm 0.195$ 	 & $0.779 \pm 0.186$ \\
$0.825 \pm 0.282$ 	 & $0.574 \pm 0.295$ 	 & $0.641 \pm 0.28$ \\


In [19]:
load_scores("labeling")
print_metrics(df_pangebin)
print_metrics(df_pbf_mean)
print_metrics(df_pbf_unicycler)
print_metrics(df_pbf_skesa)
print_metrics(df_gplas_unicycler)
print_metrics(df_gplas_skesa)


$0.718 \pm 0.297$ 	 & $0.917 \pm 0.127$ 	 & $0.764 \pm 0.238$ \\
$0.69 \pm 0.265$ 	 & $0.906 \pm 0.165$ 	 & $0.712 \pm 0.224$ \\
$0.558 \pm 0.341$ 	 & $0.927 \pm 0.148$ 	 & $0.628 \pm 0.283$ \\
$0.823 \pm 0.269$ 	 & $0.884 \pm 0.202$ 	 & $0.797 \pm 0.25$ \\
$0.963 \pm 0.112$ 	 & $0.843 \pm 0.172$ 	 & $0.882 \pm 0.155$ \\
$0.906 \pm 0.266$ 	 & $0.64 \pm 0.283$ 	 & $0.721 \pm 0.278$ \\


In [20]:

# trittico labeling

# precision plasbin-flow pangebin blue
# recall plasbin-flow pangebin verde
# f1-score plasbin0flow pangebin arancio


# df_pbf_skesa = df_labeling[(df_labeling["tool"] == "1.ske") & (df_labeling["model"] == "pbf")]
# df_pbf_skesa_precision = df_pbf_skesa[["sample", "prec"]].rename(columns={"prec": "skesa"})

# df_pbf_uni = df_labeling[(df_labeling["tool"] == "1.uni") & (df_labeling["model"] == "pbf")]
# df_pbf_uni_precision = df_pbf_uni[["sample", "prec"]].rename(columns={"prec": "uni"})

# df_pbf_precision = df_pbf_skesa_precision.merge(df_pbf_uni_precision, on="sample")
# df_pbf_precision.head()

# plt.figure(figsize=(6, 6))
# p = sns.relplot(
#     data=df_pbf_precision,
#     y="skesa",
#     x="uni",
#     kind="scatter",
# ).set(xlim=(0, 1), ylim=(0,1))


In [21]:
import matplotlib.colors as mcolors
def trittico(df_a, df_b, name_a, name_b, label, asm):
    # assert(label in ['labeling', 'binning'])
    
    cblue = mcolors.TABLEAU_COLORS["tab:blue"]
    corange = mcolors.TABLEAU_COLORS["tab:orange"]
    cgreen = mcolors.TABLEAU_COLORS["tab:green"]
    cred = mcolors.TABLEAU_COLORS["tab:red"] 

    fig = plt.figure(figsize=(15,5))
    fig.suptitle(f"{label} scores for {name_a} (x axis) against {name_b} (y axis)")


    df_prec = df_a[["sample", "prec"]].rename(columns={"prec": "A"})
    df_prec = df_prec.merge(df_b[["sample", "prec"]].rename(columns={"prec": "B"}),
                              on="sample")
    df_rec = df_a[["sample", "rec"]].rename(columns={"rec": "A"})
    df_rec = df_rec.merge(df_b[["sample", "rec"]].rename(columns={"rec": "B"}),
                              on="sample")
    df_f1 = df_a[["sample", "f1"]].rename(columns={"f1": "A"})
    df_f1 = df_f1.merge(df_b[["sample", "f1"]].rename(columns={"f1": "B"}),
                        on="sample")
    gs = fig.add_gridspec(1, 3, hspace=0, wspace=0.1)

    (prec, rec, f1) = gs.subplots(sharex='col', sharey='row')
    prec.scatter(x = df_prec["A"], y=df_prec["B"], color=cblue)
    prec.axline( (0,0),slope=1,linestyle='--',color=cred)
    prec.set_title("Precision")


    rec.scatter(x = df_rec["A"], y=df_rec["B"], color=corange)
    rec.axline( (0,0),slope=1,linestyle='--',color=cred)
    rec.set_title("Recall")

    f1.scatter(x = df_f1["A"], y=df_f1["B"], color=cgreen)
    f1.axline( (0,0),slope=1,linestyle='--',color=cred)
    f1.set_title("F1-Score")

    gs.update(left=0.07, right=0.97)
    
    fig.supxlabel(name_a)
    fig.supylabel(name_b)
    plt.savefig(f"misc/plot/{label}.{asm}.scatter.pdf")
    plt.close()

    # fig.xlabel(name_a)
    # fig.ylabel(name_b)


    

In [22]:
load_scores("labeling")
trittico(df_pangebin, df_pbf_mean, "Pangebins", "Plasbin-Flow[mean]", "Labeling", "p")
load_scores("binning")
trittico(df_pangebin, df_pbf_mean, "Pangebins", "Plasbin-Flow[mean]", "Binning", "p")
trittico(df_pangebin, df_pbf_unicycler, "Pangebins", "Plasbin-Flow [unicycler]", "Binning", "u")
trittico(df_pangebin, df_pbf_skesa, "Pangebins", "Plasbin-Flow [skesa]", "Binning", "s")


In [36]:
def doppietta(df_a, df_b, name_a, name_b, label, asm):
    # assert(label in ['labeling', 'binning'])
    
    cblue = mcolors.TABLEAU_COLORS["tab:blue"]
    corange = mcolors.TABLEAU_COLORS["tab:orange"]
    cgreen = mcolors.TABLEAU_COLORS["tab:green"]
    cred = mcolors.TABLEAU_COLORS["tab:red"] 

    fig = plt.figure(figsize=(10,5))
    fig.suptitle(f"{label} scores for {name_a} against {name_b}")


    df_prec = df_a[["sample", "prec"]].rename(columns={"prec": "A"})
    df_prec = df_prec.merge(df_b[["sample", "prec"]].rename(columns={"prec": "B"}),
                              on="sample")
    df_rec = df_a[["sample", "rec"]].rename(columns={"rec": "A"})
    df_rec = df_rec.merge(df_b[["sample", "rec"]].rename(columns={"rec": "B"}),
                              on="sample")
    
    gs = fig.add_gridspec(1, 2, hspace=0, wspace=0.1)

    (prec, rec) = gs.subplots(sharex='col', sharey='row')
    prec.scatter(x = df_prec["A"], y=df_prec["B"], color=cblue)
    prec.axline( (0,0),slope=1,linestyle='--',color=cred)
    prec.set_title("Precision")


    rec.scatter(x = df_rec["A"], y=df_rec["B"], color=corange)
    rec.axline( (0,0),slope=1,linestyle='--',color=cred)
    rec.set_title("Recall")

    gs.update(left=0.09, right=0.97)
    
    fig.supxlabel(name_a)
    fig.supylabel(name_b)
    plt.savefig(f"misc/plot/{label}.{asm}.scatter.dub.pdf")
    plt.close()

    # fig.xlabel(name_a)
    # fig.ylabel(name_b)


In [38]:
doppietta(df_pangebin, df_pbf_unicycler, "Pangebins", "Plasbin-Flow (unicycler)", "Binning", "u")
doppietta(df_pangebin, df_pbf_skesa, "Pangebins", "Plasbin-Flow (skesa)]", "Binning", "s")
doppietta(df_pangebin, df_pbf_mean,"Pangebins", "Plasbin-Flow (mean)", "Binning", "m")

In [67]:
# grid_6 
load_scores("binning")
cblue = mcolors.TABLEAU_COLORS["tab:blue"]
corange = mcolors.TABLEAU_COLORS["tab:orange"]
cgreen = mcolors.TABLEAU_COLORS["tab:green"]
cred = mcolors.TABLEAU_COLORS["tab:red"] 

fig = plt.figure(figsize=(13,17))
fig.suptitle(f"Binning scores for Pangebins against Plasbin-Flow")


# df_prec = df_a[["sample", "prec"]].rename(columns={"prec": "A"})
# df_prec = df_prec.merge(df_b[["sample", "prec"]].rename(columns={"prec": "B"}),
#                             on="sample")
# df_rec = df_a[["sample", "rec"]].rename(columns={"rec": "A"})
# df_rec = df_rec.merge(df_b[["sample", "rec"]].rename(columns={"rec": "B"}),
#                             on="sample")

gs = fig.add_gridspec(3, 2, hspace=0.1, wspace=0.1)
axss = gs.subplots(sharex='col', sharey='row')
axss[0,0].scatter(x = df_pangebin[["prec"]], y=df_pbf_skesa[["prec"]], color=cblue)
axss[0,0].axline( (0,0),slope=1,linestyle='--',color=cred)
axss[0,0].set_title("Precision (skesa)")

axss[0,1].scatter(x = df_pangebin[["rec"]], y=df_pbf_skesa[["rec"]], color=corange)
axss[0,1].axline( (0,0),slope=1,linestyle='--',color=cred)
axss[0,1].set_title("Recall (skesa)")

axss[1,0].scatter(x = df_pangebin[["prec"]], y=df_pbf_unicycler[["prec"]], color=cblue)
axss[1,0].axline( (0,0),slope=1,linestyle='--',color=cred)
axss[1,0].set_title("Precision (unicycler)")

axss[1,1].scatter(x = df_pangebin[["rec"]], y=df_pbf_unicycler[["rec"]], color=corange)
axss[1,1].axline( (0,0),slope=1,linestyle='--',color=cred)
axss[1,1].set_title("Recall (unicycler)")

axss[2,0].scatter(x = df_pangebin[["prec"]], y=df_pbf_mean[[ "prec"]], color=cblue)
axss[2,0].axline( (0,0),slope=1,linestyle='--',color=cred)
axss[2,0].set_title("Precision (skesa)")

axss[2,1].scatter(x = df_pangebin[["rec"]], y=df_pbf_mean[[ "rec"]], color=corange)
axss[2,1].axline( (0,0),slope=1,linestyle='--',color=cred)
axss[2,1].set_title("Recall (mean)")

gs.update(left=0.09, right=0.97, top=0.93, bottom=0.05)


fig.supxlabel("Pangebins")
fig.supylabel("Plasbin-Flow")
plt.savefig(f"misc/plot/big.scatter.dub.pdf")
plt.close()
